# Co-refinement of multiple contrast DMPC datasets in *refnx*

This Jupyter notebook demonstrates the utility of the *refnx* package for analysis of neutron reflectometry data. Specifically:

 - the co-refinement of three contrast variation datasets of a DMPC (1,2-dimyristoyl-sn-glycero-3-phosphocholine) bilayer measured at the solid-liquid interface with a common model
 - the use of the `LipidLeaflet` component to parameterise the model in terms of physically relevant parameters
 - the use of Bayesian Markov Chain Monte Carlo (MCMC) to investigate the Posterior distribution of the curvefitting system.
 - the intrinsic usefulness of Jupyter notebooks to facilitate reproducible research in scientific data analysis
 
 <img src="DMPC.png">
 
 The images produced in this notebook are used directly in production of the *refnx* paper.
 
 The Jupyter notebook are executable documents that can be distributed, enabling others to reproduce the data analysis contained in the document. The *refnx* documentation at https://refnx.github.io/ can be consulted for further details.

The first step in most Python scripts is to import modules and functions that are going to be used

In [ ]:
# use matplotlib for plotting
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os.path

import refnx, scipy

# the analysis module contains the curvefitting engine
from refnx.analysis import CurveFitter, Objective, Parameter, GlobalObjective, process_chain

# the reflect module contains functionality relevant to reflectometry
from refnx.reflect import SLD, ReflectModel, Structure, LipidLeaflet

# the ReflectDataset object will contain the data
from refnx.dataset import ReflectDataset

In order for the analysis to be exactly reproducible the same package versions must be used. The *conda* packaging manager, and *pip*, can be used to ensure this is the case.

In [ ]:
# version numbers used in this analysis
refnx.version.version, scipy.version.version

The `ReflectDataset` class is used to represent a dataset. They can be constructed by supplying a filename

In [ ]:
pth = os.path.join(os.path.dirname(refnx.__file__), 'analysis', 'test')

data_d2o = ReflectDataset(os.path.join(pth, 'c_PLP0016596.dat'))
data_d2o.name = "d2o"

data_hdmix = ReflectDataset(os.path.join(pth, 'c_PLP0016601.dat'))
data_hdmix.name = "hdmix"

data_h2o = ReflectDataset(os.path.join(pth, 'c_PLP0016607.dat'))
data_h2o.name = "h2o"

A `SLD` object is used to represent the Scattering Length Density of a material. It has `real` and `imag` attributes because the SLD is a complex number, with the imaginary part accounting for absorption. The units of SLD are $10^{-6} \mathring{A}^{-2}$

The `real` and `imag` attributes are `Parameter` objects. These `Parameter` objects contain the: parameter value, whether it allowed to vary, any interparameter constraints, and bounds applied to the parameter. The bounds applied to a parameter are probability distributions which encode the log-prior probability of the parameter having a certain value.

In [ ]:
si = SLD(2.07 + 0j)
sio2 = SLD(3.47 + 0j)

# the following represent the solvent contrasts used in the experiment
d2o = SLD(6.36 + 0j)
h2o = SLD(-0.56 + 0j)
hdmix = SLD(2.07 + 0j)

# We want the `real` attribute parameter to vary in the analysis, and we want to apply
# uniform bounds. The `setp` method of a Parameter is a way of changing many aspects of
# Parameter behaviour at once.
d2o.real.setp(vary=True, bounds=(6.1, 6.36))
d2o.real.name='d2o SLD'

The `LipidLeaflet` class is used to describe a single lipid leaflet in our interfacial model. A leaflet consists of a head and tail group region. Since we are studying a bilayer then inner and outer `LipidLeaflet`'s are required.

In [ ]:
# Parameter for the area per molecule each DMPC molecule occupies at the surface. We
# use the same area per molecule for the inner and outer leaflets.
apm = Parameter(56, 'area per molecule', vary=True, bounds=(54, 65))

# the sum of scattering lengths for the lipid head and tail in Angstrom.
b_heads = Parameter(6.01e-4, 'b_heads')
b_tails = Parameter(-2.92e-4, 'b_tails')

# the volume occupied by the head and tail groups in cubic Angstrom.
v_heads = Parameter(319, 'v_heads')
v_tails = Parameter(782, 'v_tails')

# the head and tail group thicknesses.
inner_head_thickness = Parameter(9, 'inner_head_thickness', vary=True, bounds=(6, 11))
outer_head_thickness = Parameter(9, 'outer_head_thickness', vary=True, bounds=(6, 11))
tail_thickness = Parameter(14, 'tail_thickness', vary=True, bounds=(12, 16))

# finally construct a `LipidLeaflet` object for the inner and outer leaflets.
# Note that here the inner and outer leaflets use the same area per molecule,
# same tail thickness, etc, but this is not necessary if the inner and outer
# leaflets are different.
inner_leaflet = LipidLeaflet(apm,
                             b_heads, v_heads, inner_head_thickness,
                             b_tails, v_tails, tail_thickness,
                             3, 3)

# we reverse the monolayer for the outer leaflet because the tail groups face upwards
outer_leaflet = LipidLeaflet(apm,
                             b_heads, v_heads, outer_head_thickness,
                             b_tails, v_tails, tail_thickness,
                             3, 0, reverse_monolayer=True)

The `Slab` Component represents a layer of uniform scattering length density of a given thickness in our interfacial model. Here we make `Slabs` from `SLD` objects, but other approaches are possible.

In [ ]:
# Slab constructed from SLD object.
sio2_slab = sio2(15, 3)
sio2_slab.thick.setp(vary=True, bounds=(10, 20))
sio2_slab.thick.name = 'sio2 thickness'
sio2_slab.rough.setp(vary=True, bounds=(1, 5))
sio2_slab.rough.name = name='sio2 roughness'
sio2_slab.vfsolv.setp(0.1, vary=True, bounds=(0., 0.5))
sio2_slab.vfsolv.name = 'sio2 solvation'

solv_roughness = Parameter(3, 'bilayer/solvent roughness')
solv_roughness.setp(vary=True, bounds=(1, 4))

Once all the `Component`s have been constructed we can chain them together to compose a `Structure` object. The `Structure` object represents the interfacial structure of our system. We create different `Structure`s for each contrast. It is important to note that each of the `Structure`s share many components, such as the `LipidLeaflet` objects. This means that parameters used to construct those components are shared between all the `Structure`s, which enables co-refinement of multiple datasets. An alternate way to carry this out would be to apply constraints to underlying parameters, but this way is clearer. Note that the final component for each structure is a `Slab` created from the solvent `SLD`s, we give those slabs a zero thickness.

In [ ]:
s_d2o = si | sio2_slab | inner_leaflet | outer_leaflet | d2o(0, solv_roughness)
s_hdmix = si | sio2_slab | inner_leaflet | outer_leaflet | hdmix(0, solv_roughness)
s_h2o = si | sio2_slab | inner_leaflet | outer_leaflet | h2o(0, solv_roughness)

The `Structure`s created in the previous step describe the interfacial structure, these structures are used to create `ReflectModel` objects that know how to apply resolution smearing, scaling factors and background.

In [ ]:
model_d2o = ReflectModel(s_d2o)
model_hdmix = ReflectModel(s_hdmix)
model_h2o = ReflectModel(s_h2o)

model_d2o.scale.setp(vary=True, bounds=(0.9, 1.1))

model_d2o.bkg.setp(vary=True, bounds=(4e-8, 5e-7))
model_hdmix.bkg.setp(vary=True, bounds=(9e-8, 5e-7))
model_h2o.bkg.setp(vary=True, bounds=(9e-8, 5e-7))

An `Objective` is constructed from a `ReflectDataset` and `ReflectModel`. Amongst other things `Objective`s can calculate chi-squared, log-likelihood probability, log-prior probability, etc. We then combine all the individual `Objective`s into a `GlobalObjective`.

In [ ]:
objective_d2o = Objective(model_d2o, data_d2o)
objective_hdmix = Objective(model_hdmix, data_hdmix)
objective_h2o = Objective(model_h2o, data_h2o)

global_objective = GlobalObjective([objective_d2o, objective_hdmix, objective_h2o])

A `CurveFitter` object can perform least squares fitting, or MCMC sampling on the `Objective` used to construct it.

In [ ]:
fitter = CurveFitter(global_objective)

We initialise the MCMC walkers by jittering around the initial position. Other modes of initialisation are possible: from a supplied covariance matrix, by sampling from the prior distributions, or by supplying known positions from an array.

In [ ]:
fitter.initialise('jitter')

In MCMC sampling a burn in period is used to allow the workers to be more representative of the distribution they are sampling. Here we do a number of samples, then discard them. The last chain position is kept to provide a starting point for the 'production' run.

In [ ]:
fitter.sample(500);
fitter.reset()

Now we do a production sampling run. We *thin* the chain as we collect, to reduce the autocorrelation of successive samples. Alternatively the thinning can be done afterwards.
In this example the total number of samples is the number of walkers (200 by default) multiplied by the number of saved samples (30): 30 * 200 = 6000. The sampling engine automatically makes full use of the total number of processing cores available to it, but this is specifiable. In addition MPI can be used, which make it useful for sampling on a cluster - MCMC is embarrasingly parallel.
Samples can be saved to file as they are acquired, useful for checkpointing sampling state.

In [ ]:
fitter.sample(30, nthin=100);

The sampling gives each varying parameter its own MCMC chain, which can be processed to give relevant statistics, or histogrammed, etc. The relationship between chains encodes the covariance of all the parameters. The chains are automatically processed to calculate the median of all the samples, and the half width of the [15.87, 84.13] percentiles. These two values are taken to be the 'fitted' parameter value, and its standard deviation. Each Parameter set to this median value, and given an `stderr` attribute.
We can see those statistics by printing the objective.

In [ ]:
print(global_objective)

Now let's see how the 'fitted' models compare to the data. We could use `global_objective.plot()`, but because we want to do a bit more tweaking for the graphics (such as vertical offsets) we're going to create the graph manually. We're also going to examine the spread in the posterior distribution.

In [ ]:
hdmix_mult = 0.001
h2o_mult = 0.1
# the data
plt.errorbar(data_d2o.x, data_d2o.y, data_d2o.y_err,
             label=data_d2o.name, ms=4, marker='o', lw=0, elinewidth=1)
plt.errorbar(data_hdmix.x, data_hdmix.y * hdmix_mult, data_hdmix.y_err * hdmix_mult,
             label=data_hdmix.name, ms=4, marker='^', lw=0, elinewidth=1)
plt.errorbar(data_h2o.x, data_h2o.y * h2o_mult, data_h2o.y_err * h2o_mult,
             label=data_h2o.name, ms=4, marker='^', lw=0, elinewidth=1)

# the median of the posterior
plt.plot(data_d2o.x, objective_d2o.generative(), color='r', zorder=20)
plt.plot(data_hdmix.x, objective_hdmix.generative() * hdmix_mult, color='r', zorder=20)
plt.plot(data_h2o.x, objective_h2o.generative() * h2o_mult, color='r', zorder=20)

# plot the spread of the fits for the different datasets
g1 = objective_d2o.pgen(500)
g2 = objective_hdmix.pgen(500)
g3 = objective_h2o.pgen(500)

save_pars = [np.array(obj.parameters) for obj in global_objective.objectives]
for i in range(500):
    plt.plot(data_d2o.x, objective_d2o.generative(next(g1)),
             color='k', alpha=0.02, zorder=10)
    plt.plot(data_hdmix.x, objective_hdmix.generative(next(g2)) * hdmix_mult,
             color='k', alpha=0.02, zorder=10)
    plt.plot(data_h2o.x, objective_h2o.generative(next(g3)) * h2o_mult,
             color='k', alpha=0.02, zorder=10)

# put back the saved parameters
for pars, objective in zip(save_pars, global_objective.objectives):
    objective.setp(pars)

plt.legend()
plt.yscale('log')
plt.ylabel('Reflectivity')
plt.xlabel('Q /$\AA^{-1}$')
plt.savefig('global_fit.png')

We can investigate the posterior distribution by a corner plot, this reveals interparameter covariances.

In [ ]:
global_objective.corner();
plt.savefig('corner.png')

The variation in scattering length density profiles can be visualised by a little bit of processing. This enables one to see what range of SLD profiles are statistically possible.

In [ ]:
saved_params = np.array(objective_d2o.parameters)

z, median_sld = s_d2o.sld_profile()

for pvec in objective_d2o.pgen(ngen=500):
    objective_d2o.setp(pvec)
    zs, sld = s_d2o.sld_profile()
    plt.plot(zs, sld, color='k', alpha=0.05)

# put back saved_params
objective_d2o.setp(saved_params)

plt.plot(z, median_sld, lw=2, color='r');
plt.ylabel('scattering length density / $10^{-6}\AA^{-2}$')
plt.xlabel('distance / $\AA$')
plt.savefig('d2o_sld_spread.png')